In [2]:
import numpy as np
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import glob
import cv2
import torch
from sklearn.cluster import KMeans
import os

In [3]:
import json

def read_poses_from_json(file_path):
    """Reads the pose data from a JSON file."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def write_matrices_to_txt(data, output_file_path):
    """Writes the 4x4 pose matrices to a text file."""
    with open(output_file_path, 'w') as file:
        for frame_id, frame_data in data.items():
            pose = frame_data['pose']
            #file.write(f"{frame_id}:\n")
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")

def write_intirinsic_to_txt(data, output_file_path):
    """Writes the 4x4 pose matrices to a text file."""
    with open(output_file_path, 'w') as file:
        for frame_id, frame_data in data.items():
            pose = frame_data['intrinsic']
            #file.write(f"{frame_id}:\n")
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")



In [4]:
import numpy as np

def quat2mat(quat):
    """Convert quaternion to rotation matrix."""
    qw, qx, qy, qz = quat
    return np.array([
        [1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
        [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
        [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]
    ])

def colmap_pose_to_arkit(R, T):
    """Convert a COLMAP world-to-camera pose to an ARKit camera-to-world pose."""
    R_inv = R.T  # Invert rotation
    T_inv = -R_inv @ T  # New translation

    # Construct the new 4x4 transformation matrix
    arkit_pose = np.eye(4)
    arkit_pose[:3, :3] = R_inv
    arkit_pose[:3, 3] = T_inv

    return arkit_pose

def process_poses(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    poses = []
    for line in lines:
        if line.startswith('#') or len(line.strip()) == 0:
            continue
        parts = line.split(' ')
        print(parts)
        qw = float(parts[1])
        qx = float(parts[2])
        qy = float(parts[3])
        qz = float(parts[4])
        tx = float(parts[5])
        ty = float(parts[6])
        tz = float(parts[7])
        camera_id = parts[8].strip()
        name = parts[9].strip()

        R = quat2mat([qw, qx, qy, qz])
        T = np.array([tx, ty, tz])

        arkit_pose = colmap_pose_to_arkit(R, T)
        poses.append(arkit_pose)
    with open(output_file, 'w') as file:
        for pose in poses:
            for row in pose:
                formatted_row = ' '.join(f"{value:.18e}" for value in row)
                file.write(formatted_row + "\n")
            file.write("\n")

# Example usage
input_file_path = '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/colmap/images.txt'  # Change this to your actual file path
output_file_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/pose.txt'  # Change this to your desired output file path

process_poses(input_file_path, output_file_path)


['0', '0.350173', '0.929442', '-0.101077', '0.0574443', '-0.413599', '1.81707', '0.290624', '1', 'frame_000000.jpg\n']
['1', '0.34196', '0.931543', '-0.107738', '0.0606927', '-0.396824', '1.818', '0.313233', '1', 'frame_000010.jpg\n']
['2', '0.332623', '0.934831', '-0.109968', '0.0579616', '-0.381526', '1.81568', '0.344949', '1', 'frame_000020.jpg\n']
['3', '0.329513', '0.937023', '-0.105328', '0.0481207', '-0.374106', '1.81771', '0.363975', '1', 'frame_000030.jpg\n']
['4', '0.335129', '0.934983', '-0.106959', '0.0453406', '-0.354452', '1.82975', '0.34419', '1', 'frame_000040.jpg\n']
['5', '0.350104', '0.930326', '-0.100609', '0.0423986', '-0.355808', '1.84828', '0.295111', '1', 'frame_000050.jpg\n']
['6', '0.371793', '0.923088', '-0.0933736', '0.0310064', '-0.339845', '1.86928', '0.224894', '1', 'frame_000060.jpg\n']
['7', '0.385091', '0.916767', '-0.1002', '0.0346787', '-0.31137', '1.88858', '0.163238', '1', 'frame_000070.jpg\n']
['8', '0.390806', '0.911408', '-0.116562', '0.0549503'

In [5]:

# Paths to your JSON input and text output files
input_json_path = '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/pose_intrinsic_imu.json'  # Change this to the path of your JSON file
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/traj.txt'  # Change this to your desired output file path

# Read the data from JSON
pose_data = read_poses_from_json(input_json_path)

# Write the matrices to a text file
write_matrices_to_txt(pose_data, output_txt_path)


pose_data = read_poses_from_json(input_json_path)
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/intrinsic.txt'
# Write the matrices to a text file
write_intirinsic_to_txt(pose_data, output_txt_path)


In [6]:
import torch.nn.functional as F
def readDepth(filepath):
    depth = cv2.imread(filepath, cv2.IMREAD_UNCHANGED)
    depth_data = depth.astype(np.float32)
    depth_data = torch.from_numpy(depth_data)
    #print(depth_data.shape)
    #print(depth_data[0,0])
    return depth_data

In [7]:

color_paths = sorted(glob.glob(f"/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/*.jpg"))
print(color_paths)

color_data = cv2.imread(color_paths[0])
image = cv2.cvtColor(color_data, cv2.COLOR_BGR2RGB)

depth_paths = sorted(glob.glob(f"/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/processedData/depth/*.png"))
print("image",image.shape)
print("DEPT",readDepth(depth_paths[0]).shape)
print(readDepth(depth_paths[0])[0][0])

['/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000000.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000001.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000002.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000003.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000004.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000005.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000006.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000007.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000008.jpg', '/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/frame_000009.jpg', '/home/ro

In [8]:
import glob
from tqdm import tqdm  # Import tqdm

def readColorImage(filepath):
    # Read color image and convert to RGB (OpenCV uses BGR by default)
    image = cv2.imread(filepath, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Convert to float32 and normalize to [0, 1] if needed
    return torch.from_numpy(image.astype(np.float32) / 255.0)

def save_depth_image(input_path, output_filepath,amount=0, upscale=1):
    """Saves the depth image to a file."""
    depth_paths = sorted(
            glob.glob(os.path.join(input_path, "*.png"))
        )
    #print(depth_paths)
    os.makedirs(output_filepath, exist_ok=True)  # Ensure output path exists
    depth_paths=depth_paths[:amount]
    for i, depth_path in enumerate(tqdm(depth_paths, desc='Upsampling depth images')):
        # this changes the vals
        depth_data=readDepth(depth_path)#*1000
        # this image shape
        upsampled_depth_bilinear = F.interpolate(depth_data.unsqueeze(0).unsqueeze(0), 
                                                scale_factor=upscale, 
                                                mode='nearest', 
                                               ).squeeze()
        #print(upsampled_depth_bilinear.shape)
        upsampled_depth_bilinear = upsampled_depth_bilinear
        #upsampled_depth = upsampled_depth_bilinear * 6553.5
        upsampled_depth = upsampled_depth_bilinear.numpy().astype(np.uint16)
        #print((upsampled_depth.shape))

        cv2.imwrite(os.path.join(output_filepath, f"frame_{str(i).zfill(6)}.png"), upsampled_depth)
        
#bilateral filter
def save_color_image(input_path, output_filepath, amount=0, upscale=1):
    """Saves the color images to a file after upscaling."""
    color_paths = sorted(glob.glob(os.path.join(input_path, "*.jpg")))
    color_paths = color_paths[:amount] if amount > 0 else color_paths
    
    os.makedirs(output_filepath, exist_ok=True)  # Ensure output path exists
    
    for i, color_path in enumerate(tqdm(color_paths, desc='Processing color images')):
        color_data = readColorImage(color_path)
        color_data = color_data.permute(2, 0, 1).unsqueeze(0)  # CxHxW and add batch dimension
        
        upsampled_color_bilinear = F.interpolate(color_data, 
                                                scale_factor=upscale, 
                                                mode='nearest', 
                                                ).squeeze(0)
        
        # Convert back to uint8 and BGR for saving
        upsampled_color = (upsampled_color_bilinear.permute(1, 2, 0) * 255).byte().numpy()
        upsampled_color = cv2.cvtColor(upsampled_color, cv2.COLOR_RGB2BGR)
        #print(upsampled_color.shape)
        cv2.imwrite(os.path.join(output_filepath, f"frame_{str(i).zfill(6)}.jpg"), upsampled_color)

#save_color_image('/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/rgb/', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/',amount=2000,upscale=0.333334)
save_depth_image('/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/depth', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/',amount=2000,upscale=2.5)
#save_depth_image('/home/rozenberszki/D_Project/wsnsl/Dataset/depth_rendered/56a0ec536c/iphone/render_depth', '/home/rozenberszki/D_Project/wsnsl/Dataset/renderedDepthScaled/',amount=946,upscale=0.333334)
#0.5, 3.75

Upsampling depth images:   1%|▏         | 28/2000 [00:00<00:07, 278.54it/s]

Upsampling depth images: 100%|██████████| 2000/2000 [00:08<00:00, 234.42it/s]


In [7]:

color_paths = sorted(glob.glob(f"/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/*.jpg"))
print(color_paths)

color_data = cv2.imread(color_paths[0])
image = cv2.cvtColor(color_data, cv2.COLOR_BGR2RGB)

depth_paths = sorted(glob.glob(f"/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/*.png"))
print("image",image.shape)
print("DEPT",readDepth(depth_paths[0]).shape)
print(readDepth(depth_paths[0])[0][0])


['/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000000.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000001.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000002.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000003.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000004.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000005.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000006.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000007.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000008.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000009.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000010.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000011.jpg', '/home/rozenberszki/D_Project/wsnsl/Dataset/color_path/frame_000012.jpg', '/home/rozenberszki/D_Project/wsnsl/D


def pose_matrix_from_quaternion( pvec):
    """convert 4x4 pose matrix to (t, q)"""
    from scipy.spatial.transform import Rotation

    pose = np.eye(4)
    pose[:3, :3] = Rotation.from_quat(pvec[3:]).as_matrix()
    pose[:3, 3] = pvec[:3]
    print(pose)
    return pose

def parse_list( filepath, skiprows=0):
    """read list data"""
    data = np.loadtxt(filepath, delimiter=" ", dtype=np.unicode_, skiprows=skiprows)
    return data


pose_list="/home/rozenberszki/project/wsnsl/Datasets/Scannet++/data/56a0ec536c/iphone/colmap/images.txt"
pose_data = parse_list(pose_list, skiprows=0)
print(pose_data.shape)
print(pose_data[0,1:-1])
pose_data_temp = np.zeros((pose_data.shape[0], 4,4))
for i in range(pose_data.shape[0]):
    pose_data_temp[i] = pose_matrix_from_quaternion(pose_data[i,1:-2])
output_txt_path = '/home/rozenberszki/D_Project/wsnsl/Dataset/pose.txt'
np.savetxt(output_txt_path, pose_data_temp.reshape(-1, pose_data_temp.shape[-1]), fmt='%f')

np.linalg.inv(pose_data_temp[0])